<a href="https://colab.research.google.com/github/PawanKrGunjan/Natural-Language-Processing/blob/main/Audio/Build_a_music_genre_classifier_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets[audio]
!pip install transformers[torch]
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 28.4 MB/s eta 0:00:00


# Load the dataset

In [2]:
from datasets import load_dataset

gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 999
    })
})

# Split into train & test

In [3]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.1)
gtzan

DatasetDict({
    train: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 899
    })
    test: Dataset({
        features: ['file', 'audio', 'genre'],
        num_rows: 100
    })
})

In [4]:
id2label_fn = gtzan["train"].features["genre"].int2str
id2label_fn

<bound method ClassLabel.int2str of ClassLabel(names=['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz', 'metal', 'pop', 'reggae', 'rock'], id=None)>

# Load the Automatic Classification Pipeline

In [5]:
from transformers import pipeline

classifier = pipeline(
    "audio-classification",
    model='PawanKrGunjan/whisper-tiny-finetuned-gtzan',
)

In [6]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.10720825,  0.16122437,  0.28585815, ..., -0.22924805,
         -0.20629883, -0.11334229]),
  'sampling_rate': 22050},
 'genre': 7}

In [7]:
gtzan["train"][0]['audio']['sampling_rate']

22050

In [8]:
from IPython.display import Audio
import librosa


# Load the audio file using librosa or any other audio processing library you prefer
audio_path=gtzan["train"][0]['audio']['path']
sample_rate =gtzan["train"][0]['audio']['sampling_rate']

audio_data, _ = librosa.load(audio_path, sr=sample_rate)

# Display the audio using IPython.display.Audio
display(Audio(data=audio_data, rate=sample_rate))

In [9]:
pred_genre = classifier(gtzan["train"][0]['audio']['path'])
pred_genre

[{'score': 0.9450774788856506, 'label': 'pop'},
 {'score': 0.049123365432024, 'label': 'disco'},
 {'score': 0.0044833458960056305, 'label': 'reggae'},
 {'score': 0.0012680547079071403, 'label': 'hiphop'},
 {'score': 3.0037315809749998e-05, 'label': 'rock'}]

In [10]:
id2label_fn(gtzan["train"][0]["genre"])

actual_genre = gtzan["train"][0]["genre"]

print('Predicted Genre: ',pred_genre[0]['label'].split('_')[-1])
print('Actual Genre :',id2label_fn(gtzan["train"][0]["genre"]))

Predicted Genre:  pop
Actual Genre : pop


# Load the transformer AutoFeatureExtractor

In [11]:
from transformers import AutoFeatureExtractor

model_id = "anton-l/distilhubert-ft-keyword-spotting"
model_id ='openai/whisper-tiny'
model_id ='openai/whisper-base'
model_id ='PawanKrGunjan/whisper-tiny-finetuned-gtzan'
model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

### Sampling rate

In [12]:
sampling_rate = feature_extractor.sampling_rate
sampling_rate

16000

In [13]:
from datasets import Audio

gtzan = gtzan.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [14]:
gtzan["train"][0]

{'file': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/5022b0984afa7334ff9a3c60566280b08b5179d4ac96a628052bada7d8940244/genres/pop/pop.00098.wav',
  'array': array([ 0.0873509 ,  0.20183384,  0.4790867 , ..., -0.18743178,
         -0.23294401, -0.13517427]),
  'sampling_rate': 16000},
 'genre': 7}

### Mean & Variance

In [15]:
import numpy as np

sample = gtzan["train"][0]["audio"]

print(f"Mean: {np.mean(sample['array']):.3}, Variance: {np.var(sample['array']):.3}")

Mean: 0.000185, Variance: 0.0493


#### After the feature extractor

In [17]:
inputs = feature_extractor(sample["array"], sampling_rate=sample["sampling_rate"])

print(f"inputs keys: {list(inputs.keys())}")

print(
    f"Mean: {np.mean(inputs['input_values']):.3}, Variance: {np.var(inputs['input_values']):.3}"
)

inputs keys: ['input_values', 'attention_mask']
Mean: -7.45e-09, Variance: 1.0


# Preprocessing

In [18]:
max_duration = 30.0


def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [19]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=32,
    num_proc=1,
)
gtzan_encoded

Map:   0%|          | 0/899 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 899
    })
    test: Dataset({
        features: ['genre', 'input_values', 'attention_mask'],
        num_rows: 100
    })
})

### Change the name 'genre' to 'label' of dataset

In [20]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

### Id to label & Label to id

In [21]:
id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

id2label

{'0': 'blues',
 '1': 'classical',
 '2': 'country',
 '3': 'disco',
 '4': 'hiphop',
 '5': 'jazz',
 '6': 'metal',
 '7': 'pop',
 '8': 'reggae',
 '9': 'rock'}

# Load the model

In [22]:
from transformers import AutoModelForAudioClassification

num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True
)

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['projector.bias', 'classifier.weight', 'classifier.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Hugging Face login

In [23]:
from huggingface_hub import notebook_login
notebook_login()

# Training Arguments

In [28]:
from transformers import TrainingArguments

# Define other required variables
model_name = model_id.split("/")[-1]
batch_size = 8
gradient_accumulation_steps = 1
num_train_epochs = 10

# Define TrainingArguments
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-05,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=0.1,
    logging_steps=5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,
    push_to_hub=True,
    seed=42,  # Added seed
    save_total_limit=1,  # Save only the best model
    lr_scheduler_type="linear",  # Linear learning rate scheduler
)

## Evaluations

In [29]:
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

## *Optimizer*

In [30]:
import torch
from transformers import get_linear_schedule_with_warmup

# Create PyTorch AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), betas=(0.9, 0.999), eps=1e-8)

# Create linear learning rate scheduler with warmup
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=int(num_train_epochs * len(gtzan_encoded["train"]) / batch_size * 0.1),
                                            num_training_steps=num_train_epochs * len(gtzan_encoded["train"]) / batch_size)

# Trainer

In [31]:
from transformers import Trainer

# Create Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, scheduler),  # Pass optimizer and scheduler
)

# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,2.297400,2.369697,0.150000
2,1.844200,2.070132,0.230000
3,1.832700,1.790918,0.370000
4,1.918700,1.633498,0.480000
5,1.542300,nan,0.430000
6,1.442100,1.721532,0.400000
7,1.945900,1.588558,0.450000
8,1.215600,nan,0.480000
9,1.484600,nan,0.530000
10,0.939000,nan,0.510000


TrainOutput(global_step=1130, training_loss=1.6923485629326473, metrics={'train_runtime': 5958.3961, 'train_samples_per_second': 1.509, 'train_steps_per_second': 0.19, 'total_flos': 6.133988274624e+17, 'train_loss': 1.6923485629326473, 'epoch': 10.0})

# Save to hub

In [32]:
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)

'https://huggingface.co/PawanKrGunjan/distilhubert-finetuned-gtzan/tree/main/'